In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [13]:
x_train.shape[0]

60000

In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_height//(max_pool1_k*max_pool2_k))*(input_width//(max_pool1_k*max_pool2_k)) * n_conv2

In [5]:
#Very important to know how many biases and how many weights are there in each layer
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden,n_out]))                  
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out]))
}

In [6]:
def cnn(x,weights,biases,keep_prob):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    #-1 basically means that tensorflow will calculate the remaining dimension again
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    
    hidden_output = tf.nn.dropout(hidden_output_before_dropout,keep_prob)
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [7]:
def conv(x,weights,bias,stride=1):
    out = tf.nn.conv2d(x,weights,padding="SAME",strides=[1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize = [1,k,k,1],strides=[1,k,k,1])

In [8]:
x = tf.compat.v1.placeholder("float",[None,input_pixels])
y = tf.compat.v1.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x,weights,biases,keep_prob)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

In [10]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [11]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [17]:
#giving an error because i'm using tensorflow 2 and the following syntax is in tensorflow 1
batch_size = 100
for i in range(25):
    num_batches = int(x_train[0]/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y,keep_prob:0.8})
        total_cost += c
    print(total_cost)

ValueError: Cannot feed value of shape (100, 28, 28) for Tensor 'Placeholder:0', which has shape '(?, 784)'

In [15]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels, keep_prob:1.0})
correct_preds.sum()

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 